In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

 # Autorater using `CustomMetric`
* [Bring-Your-Own-Autorater using CustomMetric | Gen AI Evaluation SDK Tutorial](https://colab.sandbox.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/bring_your_own_autorater_with_custom_metric.ipynb#scrollTo=w-Edsbk7JRim)           

In [1]:
# @title Install Vertex AI Python SDK
%pip install --upgrade --user --quiet google-cloud-aiplatform[evaluation]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.5 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# @title Define constants

PROJECT_ID="ai-hangsik" # @param {type:"string"}
LOCATION="us-central1" # @param {type:"string"}
EXPERIMENT_NAME = "my-eval-task-experiment"  # @param {type:"string"}

In [3]:
# @title GCP Authentication

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)

### Set Google Cloud project information and initialize Vertex AI SDK

In [4]:
# @title Initialize Vertex AI SDK
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [5]:
# General
import inspect
import json
import logging
import warnings

from IPython.display import HTML, Markdown, display
import pandas as pd
import plotly.graph_objects as go

# Main
from vertexai.evaluation import CustomMetric, EvalTask
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
)

### Helper functions

In [6]:
def print_doc(function):
    print(f"{function.__name__}:\n{inspect.getdoc(function)}\n")


def display_eval_result(eval_result, title=None, metrics=None):
    """Display the evaluation results."""
    summary_metrics, metrics_table = (
        eval_result.summary_metrics,
        eval_result.metrics_table,
    )

    metrics_df = pd.DataFrame.from_dict(summary_metrics, orient="index").T
    if metrics:
        metrics_df = metrics_df.filter(
            [
                metric
                for metric in metrics_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )
        metrics_table = metrics_table.filter(
            [
                metric
                for metric in metrics_table.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    if title:
        # Display the title with Markdown for emphasis
        display(Markdown(f"## {title}"))
    # Display the summary metrics DataFrame
    display(Markdown("### Summary Metrics"))
    display(metrics_df)
    # Display the metrics table DataFrame
    display(Markdown("### Row-based Metrics"))
    display(metrics_table)


def display_explanations(eval_result, metrics=None, n=1):
    """Display the explanations."""
    style = "white-space: pre-wrap; width: 1500px; overflow-x: auto;"
    metrics_table = eval_result.metrics_table
    df = metrics_table.sample(n=n)

    if metrics:
        df = df.filter(
            ["response", "baseline_model_response"]
            + [
                metric
                for metric in df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )
    for index, row in df.iterrows():
        for col in df.columns:
            display(HTML(f"<div style='{style}'><h4>{col}:</h4>{row[col]}</div>"))
        display(HTML("<hr>"))


def plot_radar_plot(eval_results, max_score=5, metrics=None):
    fig = go.Figure()

    for eval_result in eval_results:
        title, summary_metrics, report_df = eval_result

        if metrics:
            summary_metrics = {
                k: summary_metrics[k]
                for k, v in summary_metrics.items()
                if any(selected_metric in k for selected_metric in metrics)
            }

        fig.add_trace(
            go.Scatterpolar(
                r=list(summary_metrics.values()),
                theta=list(summary_metrics.keys()),
                fill="toself",
                name=title,
            )
        )

    fig.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, max_score])), showlegend=True
    )

    fig.show()


def display_radar_plot(eval_results, metrics=None):
    """Plot the radar plot."""
    fig = go.Figure()
    for item in eval_results:
        title, eval_result = item
        summary_metrics = eval_result.summary_metrics
        if metrics:
            summary_metrics = {
                k.replace("/mean", ""): summary_metrics[k]
                for k, v in summary_metrics.items()
                if any(selected_metric + "/mean" in k for selected_metric in metrics)
            }
        fig.add_trace(
            go.Scatterpolar(
                r=list(summary_metrics.values()),
                theta=list(summary_metrics.keys()),
                fill="toself",
                name=title,
            )
        )
    fig.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, 5])), showlegend=True
    )
    fig.show()


def display_bar_plot(eval_results_list, metrics=None):
    fig = go.Figure()
    data = []

    for eval_results in eval_results_list:
        title, eval_result = eval_results[0], eval_results[1]

        summary_metrics = eval_result.summary_metrics
        mean_summary_metrics = [f"{metric}/mean" for metric in metrics]
        updated_summary_metrics = []
        if metrics:
            for k, v in summary_metrics.items():
                if k in mean_summary_metrics:
                    updated_summary_metrics.append((k, v))
            summary_metrics = dict(updated_summary_metrics)
            # summary_metrics = {k: summary_metrics[k] for k, v in summary_metrics.items() if any(selected_metric in k for selected_metric in metrics)}

        data.append(
            go.Bar(
                x=list(summary_metrics.keys()),
                y=list(summary_metrics.values()),
                name=title,
            )
        )

    fig = go.Figure(data=data)

    # Change the bar mode
    fig.update_layout(barmode="group", showlegend=True)
    fig.show()


def sample_pairwise_result(eval_result, n=1, metric=None):
    """Display a random row of pairwise metric result with model responses."""
    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = eval_result.metrics_table
    df = df.sample(n=n)
    for index, row in df.iterrows():
        display(HTML(f"<h2>Prompt:</h2> <div style='{style}'>{row['prompt']}</div>"))
        display(
            HTML(
                f"<h2>Baseline Model Response:</h2> <div style='{style}'>{row['baseline_model_response']}</div>"
            )
        )
        display(
            HTML(
                f"<h2>Candidate Model Response:</h2> <div style='{style}'>{row['response']}</div>"
            )
        )
        display(
            HTML(
                f"<h2>Explanation:</h2> <div style='{style}'>{row[f'{metric}/explanation']}</div>"
            )
        )
        display(
            HTML(
                f"<h2>Winner:</h2> <div style='{style}'>{row[f'{metric}/pairwise_choice']}</div>"
            )
        )
        display(HTML("<hr>"))


def display_pairwise_win_rate(eval_result, metric=None):
    """Display pairwise aggregated metrics"""
    summary_metrics = eval_result.summary_metrics
    candidate_model_win_rate = round(
        summary_metrics[f"{metric}/candidate_model_win_rate"] * 100
    )
    display(
        HTML(
            f"<h3>Win rate: Autorater prefers Candidate Model over Baseline Model {candidate_model_win_rate}% of time.</h3>"
        )
    )

## Using locally-defined `CustomMetric` class


### Documentation


In [7]:
print_doc(CustomMetric)

CustomMetric:
The custom evaluation metric.

A fully-customized CustomMetric that can be used to evaluate a single model
by defining a metric function for a computation-based metric. The
CustomMetric is computed on the client-side using the user-defined metric
function in SDK only, not by the Vertex Gen AI Evaluation Service.

  Attributes:
    name: The name of the metric.
    metric_function: The user-defined evaluation function to compute a metric
      score. Must use the dataset row dictionary as the metric function
      input and return per-instance metric result as a dictionary output.
      The metric score must mapped to the name of the CustomMetric as key.



### Define a `CustomMetric` and "Bring-Your-Own-Autorater"

Define a custom model-based metric and "Bring-Your-Own-Autorater". The custom model-based metric can contain multiple fields such as an autorater-graded score, explanations for the score, and more. The score field must match the name of the metric, other fields can be freeform strings.

In [8]:
# Bring-Your-Own-Autorater and define your autorater inference function


def get_autorater_response(metric_prompt: str) -> dict:
    metric_response_schema = {
        "type": "OBJECT",
        "properties": {
            "score": {"type": "NUMBER"},
            "explanation": {"type": "STRING"},
        },
        "required": ["score", "explanation"],
    }

    autorater = GenerativeModel(
        "gemini-1.5-pro",
        generation_config=GenerationConfig(
            response_mime_type="application/json",
            response_schema=metric_response_schema,
        ),
        safety_settings={
            HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        },
    )

    response = autorater.generate_content(metric_prompt)

    response_json = {}

    if response.candidates and len(response.candidates) > 0:
        candidate = response.candidates[0]
        if (
            candidate.content
            and candidate.content.parts
            and len(candidate.content.parts) > 0
        ):
            part = candidate.content.parts[0]
            if part.text:
                response_json = json.loads(part.text)

    return response_json

In [9]:
# Define your metric computation function


def linguistic_acceptability_fn(instance: dict) -> dict:
    metric_prompt_template = """
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by AI models. We will provide you with the user prompt and an AI-generated responses.
You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the Criteria provided in the Evaluation section below.
You will assign the response a rating following the Rating Rubric and Evaluation Steps. Give step by step explanations for your rating, and only choose ratings from the Rating Rubric.


# Evaluation
## Criteria
Appropriate word choice: Words chosen are appropriate and purposeful given their relative context and positioning in the text. Vocabulary demonstrates prompt understanding.
Proper Grammar: The language's grammar rules are correctly followed, including but not limited to sentence structures, verb tenses, subject-verb agreement, proper punctuation, and capitalization.
Reference Alignment: The response is consistent and aligned with the reference.

## Rating Rubric
1: Poor: The writing is riddled with grammatical errors, uses highly inappropriate vocabulary, is completely unrelated to the reference.
2: Unsatisfactory: The writing has significant grammatical errors, uses inappropriate vocabulary, deviates significantly from the reference.
3: Satisfactory: The writing may have minor grammatical errors or use less-appropriate vocabulary, but it aligns reasonably well with the reference.
4: Good: The writing is generally grammatically correct, uses appropriate vocabulary and aligns well with the reference.
5: Excellent: The writing is grammatically correct, uses appropriate vocabulary and aligns perfectly with the reference.

## Evaluation Steps
Step 1: Assess the response in aspects of all criteria provided. Provide assessment according to each criterion.
Step 2: Score based on the rating rubric. Give a brief rationale to explain your evaluation considering each individual criterion.


# User Inputs and AI-generated Response
## User Inputs
### prompt
{prompt}

## AI-generated Response
{response}
"""
    prompt = instance["prompt"]
    response = instance["response"]

    metric_prompt = metric_prompt_template.format(prompt=prompt, response=response)

    response_dict = get_autorater_response(metric_prompt)

    return {
        "linguistic_acceptability": response_dict["score"],
        "explanation": response_dict["explanation"],
    }

In [10]:
# Define a CustomMetric
linguistic_acceptability_metric = CustomMetric(
    name="linguistic_acceptability",
    metric_function=linguistic_acceptability_fn,
)

### Run Evaluation with `CustomMetric`

* The registered custom metrics are computed on the client side, without using Vertex Gen AI Evaluation Service APIs.

* Mixing CustomMetric instances and other types of metrics is supported.

* You can handle rate limiting of your own autorater with Eval SDK's [RateLimiter utils function](https://github.com/googleapis/python-aiplatform/blob/20f2cad85719b4793c2b77bf36c5a0d775b1a825/vertexai/evaluation/utils.py#L104) based on your autorater's capacity and QPS limit.


#### Define an evaluation dataset

In [11]:
instruction = "Summarize the following article"

context = [
    "To make a classic spaghetti carbonara, start by bringing a large pot of salted water to a boil. While the water is heating up, cook pancetta or guanciale in a skillet with olive oil over medium heat until it's crispy and golden brown. Once the pancetta is done, remove it from the skillet and set it aside. In the same skillet, whisk together eggs, grated Parmesan cheese, and black pepper to make the sauce. When the pasta is cooked al dente, drain it and immediately toss it in the skillet with the egg mixture, adding a splash of the pasta cooking water to create a creamy sauce.",
    "Preparing a perfect risotto requires patience and attention to detail. Begin by heating butter in a large, heavy-bottomed pot over medium heat. Add finely chopped onions and minced garlic to the pot, and cook until they're soft and translucent, about 5 minutes. Next, add Arborio rice to the pot and cook, stirring constantly, until the grains are coated with the butter and begin to toast slightly. Pour in a splash of white wine and cook until it's absorbed. From there, gradually add hot chicken or vegetable broth to the rice, stirring frequently, until the risotto is creamy and the rice is tender with a slight bite.",
    "For a flavorful grilled steak, start by choosing a well-marbled cut of beef like ribeye or New York strip. Season the steak generously with kosher salt and freshly ground black pepper on both sides, pressing the seasoning into the meat. Preheat a grill to high heat and brush the grates with oil to prevent sticking. Place the seasoned steak on the grill and cook for about 4-5 minutes on each side for medium-rare, or adjust the cooking time to your desired level of doneness. Let the steak rest for a few minutes before slicing against the grain and serving.",
    "Creating a creamy homemade tomato soup is a comforting and simple process. Begin by heating olive oil in a large pot over medium heat. Add diced onions and minced garlic to the pot and cook until they're soft and fragrant. Next, add chopped fresh tomatoes, chicken or vegetable broth, and a sprig of fresh basil to the pot. Simmer the soup for about 20-30 minutes, or until the tomatoes are tender and falling apart. Remove the basil sprig and use an immersion blender to puree the soup until smooth. Season with salt and pepper to taste before serving.",
    "To bake a decadent chocolate cake from scratch, start by preheating your oven to 350°F (175°C) and greasing and flouring two 9-inch round cake pans. In a large mixing bowl, cream together softened butter and granulated sugar until light and fluffy. Beat in eggs one at a time, making sure each egg is fully incorporated before adding the next. In a separate bowl, sift together all-purpose flour, cocoa powder, baking powder, baking soda, and salt. Divide the batter evenly between the prepared cake pans and bake for 25-30 minutes, or until a toothpick inserted into the center comes out clean.",
]

reference = [
    "The process of making spaghetti carbonara involves boiling pasta, crisping pancetta or guanciale, whisking together eggs and Parmesan cheese, and tossing everything together to create a creamy sauce.",
    "Preparing risotto entails sautéing onions and garlic, toasting Arborio rice, adding wine and broth gradually, and stirring until creamy and tender.",
    "Grilling a flavorful steak involves seasoning generously, preheating the grill, cooking to desired doneness, and letting it rest before slicing.",
    "Creating homemade tomato soup includes sautéing onions and garlic, simmering with tomatoes and broth, pureeing until smooth, and seasoning to taste.",
    "Baking a decadent chocolate cake requires creaming butter and sugar, beating in eggs and alternating dry ingredients with buttermilk before baking until done.",
]


eval_dataset = pd.DataFrame(
    {
        "context": context,
        "instruction": [instruction] * len(context),
        "reference": reference,
    }
)

#### Define prompt templates to compare


In [12]:
prompt_templates = [
    "Instruction: {instruction}. Article: {context}. Summary:",
    "Article: {context}. Complete this task: {instruction}, in three sentence. Summary:",
    "Goal: {instruction} and give me a TLDR. Here's a news article: {context}. Summary:",
]

#### Define metrics


In [13]:
metrics = ["rouge_l_sum", "fluency", "coherence", linguistic_acceptability_metric]

#### Define an EvalTask

In [14]:
generation_config = GenerationConfig(
    temperature=0.3,
    top_k=1,
)

safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

gemini_model = GenerativeModel(
    "gemini-1.5-pro",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

In [15]:
experiment_name = "eval-custom-metric"  # @param {type:"string"}

eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=metrics,
    experiment=experiment_name,
)

#### Running Evaluation

In [16]:
eval_results = []

for i, prompt_template in enumerate(prompt_templates):
    eval_result = eval_task.evaluate(
        model=gemini_model,
        prompt_template=prompt_template,
    )

    eval_results.append((f"Prompt #{i}", eval_result))

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/eval-custom-metric-599561ac-d7cb-41f5-a520-420f70aea802 to Experiment: eval-custom-metric


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': 'Instruction: {instruction}. Article: {context}. Summary:', 'model_name': 'publishers/google/models/gemini-1.5-pro', 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-1.5-pro.
100%|██████████| 5/5 [00:02<00:00,  2.26it/s]
INFO:vertexai.evaluation._evaluation:All 5 responses are successfully generated from Gemini model gemini-1.5-pro.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 2.2282356850000156 seconds.
IN

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/eval-custom-metric-440ae41a-66df-49ed-b3ce-676504ede62e to Experiment: eval-custom-metric


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': 'Article: {context}. Complete this task: {instruction}, in three sentence. Summary:', 'model_name': 'publishers/google/models/gemini-1.5-pro', 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-1.5-pro.
100%|██████████| 5/5 [00:02<00:00,  2.36it/s]
INFO:vertexai.evaluation._evaluation:All 5 responses are successfully generated from Gemini model gemini-1.5-pro.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 2.12

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/eval-custom-metric-ee50e0b4-23c6-4236-9fb7-e9ce412ac489 to Experiment: eval-custom-metric


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': "Goal: {instruction} and give me a TLDR. Here's a news article: {context}. Summary:", 'model_name': 'publishers/google/models/gemini-1.5-pro', 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-1.5-pro.
100%|██████████| 5/5 [00:03<00:00,  1.53it/s]
INFO:vertexai.evaluation._evaluation:All 5 responses are successfully generated from Gemini model gemini-1.5-pro.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 3.27

#### Display Evaluation result and explanations

In [17]:
for title, eval_result in eval_results:
    display_eval_result(eval_result, title=title)

## Prompt #0

### Summary Metrics

,row_count,rouge_l_sum/mean,rouge_l_sum/std,fluency/mean,fluency/std,coherence/mean,coherence/std,linguistic_acceptability/mean,linguistic_acceptability/std
0,5.0,0.355634,0.069161,4.8,0.447214,4.8,0.447214,4.6,0.547723


### Row-based Metrics

,context,instruction,reference,prompt,response,linguistic_acceptability/score,linguistic_acceptability/explanation,rouge_l_sum/score,fluency/explanation,fluency/score,coherence/explanation,coherence/score
0,"To make a classic spaghetti carbonara, start b...",Summarize the following article,The process of making spaghetti carbonara invo...,Instruction: Summarize the following article. ...,This article provides a concise guide to makin...,5.0,The response provides an accurate and fluent s...,0.315789,STEP 1: Assess grammar correctness: The respon...,5.0,STEP 1: The purpose of the text is to summariz...,5.0
1,Preparing a perfect risotto requires patience ...,Summarize the following article,Preparing risotto entails sautéing onions and ...,Instruction: Summarize the following article. ...,Making perfect risotto is a labor of love! Sta...,4.0,Appropriate word choice: The response uses fit...,0.313253,STEP 1: Assess grammar correctness: The respon...,5.0,STEP 1: The purpose of the response is to summ...,5.0
2,"For a flavorful grilled steak, start by choosi...",Summarize the following article,Grilling a flavorful steak involves seasoning ...,Instruction: Summarize the following article. ...,This article provides a concise guide to grill...,5.0,Appropriate word choice: The response uses cle...,0.329412,STEP 1: Assess grammar correctness: The respon...,5.0,STEP 1: The purpose is to summarize the provid...,5.0
3,Creating a creamy homemade tomato soup is a co...,Summarize the following article,Creating homemade tomato soup includes sautéin...,Instruction: Summarize the following article. ...,This article provides a simple recipe for crea...,5.0,This response is excellent. It accurately summ...,0.477612,STEP 1: Assess grammar correctness: The respon...,5.0,STEP 1: The purpose is to summarize the given ...,5.0
4,To bake a decadent chocolate cake from scratch...,Summarize the following article,Baking a decadent chocolate cake requires crea...,Instruction: Summarize the following article. ...,This article provides a concise guide to bakin...,4.0,Appropriate word choice: The response uses wor...,0.342105,STEP 1: Assess grammar correctness: The respon...,4.0,STEP 1: The purpose of the provided text is to...,4.0


...

## Prompt #2

### Summary Metrics

,row_count,rouge_l_sum/mean,rouge_l_sum/std,fluency/mean,fluency/std,coherence/mean,coherence/std,linguistic_acceptability/mean,linguistic_acceptability/std
0,5.0,0.259069,0.037044,5.0,0.0,5.0,0.0,4.2,0.83666


### Row-based Metrics

,context,instruction,reference,prompt,response,linguistic_acceptability/score,linguistic_acceptability/explanation,rouge_l_sum/score,fluency/explanation,fluency/score,coherence/explanation,coherence/score
0,"To make a classic spaghetti carbonara, start b...",Summarize the following article,The process of making spaghetti carbonara invo...,Goal: Summarize the following article and give...,## Classic Spaghetti Carbonara: A Quick Summar...,5,The response provides an excellent summary of ...,0.218182,STEP 1: Grammar Correctness: The AI model's re...,5.0,STEP 1: The purpose is to summarize a recipe f...,5.0
1,Preparing a perfect risotto requires patience ...,Summarize the following article,Preparing risotto entails sautéing onions and ...,Goal: Summarize the following article and give...,## Risotto Rundown: \n\nThis article outlines ...,4,The response provides an accurate and well-wri...,0.263566,STEP 1: Assess grammar correctness: The respon...,5.0,STEP 1: The purpose is to summarize an article...,5.0
2,"For a flavorful grilled steak, start by choosi...",Summarize the following article,Grilling a flavorful steak involves seasoning ...,Goal: Summarize the following article and give...,## Grilled Steak Summary:\n\nThis article prov...,5,The response provides an accurate and well-wri...,0.242991,STEP 1: Assess grammar correctness: The respon...,5.0,STEP 1: The purpose is to summarize provided i...,5.0
3,Creating a creamy homemade tomato soup is a co...,Summarize the following article,Creating homemade tomato soup includes sautéin...,Goal: Summarize the following article and give...,## Creamy Tomato Soup: From Stove to Bowl\n\nT...,3,Appropriate word choice: The word choice is ap...,0.252427,STEP 1: Assess grammar correctness: The respon...,5.0,STEP 1: The purpose of the AI response is to s...,5.0
4,To bake a decadent chocolate cake from scratch...,Summarize the following article,Baking a decadent chocolate cake requires crea...,Goal: Summarize the following article and give...,## Decadent Chocolate Cake Recipe Summary:\n\n...,4,Appropriate word choice: The response uses wor...,0.318182,STEP 1: Assess grammar correctness: The AI-gen...,5.0,STEP 1: The purpose of the prompt is to create...,5.0


In [18]:
for title, eval_result in eval_results:
    display_explanations(eval_result, metrics=["linguistic_acceptability"])

#### Compare Eval Results

In [19]:
display_radar_plot(
    eval_results,
    metrics=["fluency", "coherence", "linguistic_acceptability"],
)

In [20]:
display_bar_plot(eval_results, metrics=["rouge_l_sum"])

In [21]:
eval_task.display_runs()

,experiment_name,run_name,run_type,state,param.model_name,param.HARM_CATEGORY_HATE_SPEECH,param.prompt_template,param.HARM_CATEGORY_HARASSMENT,param.HARM_CATEGORY_DANGEROUS_CONTENT,param.HARM_CATEGORY_UNSPECIFIED,param.HARM_CATEGORY_SEXUALLY_EXPLICIT,metric.row_count,metric.fluency/std,metric.coherence/std,metric.linguistic_acceptability/std,metric.linguistic_acceptability/mean,metric.fluency/mean,metric.coherence/mean,metric.rouge_l_sum/mean,metric.rouge_l_sum/std
0,eval-custom-metric,ee50e0b4-23c6-4236-9fb7-e9ce412ac489,system.ExperimentRun,COMPLETE,publishers/google/models/gemini-1.5-pro,BLOCK_NONE,Goal: {instruction} and give me a TLDR. Here's...,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,5.0,0.000000,0.000000,0.836660,4.2,5.0,5.0,0.259069,0.037044
1,eval-custom-metric,440ae41a-66df-49ed-b3ce-676504ede62e,system.ExperimentRun,COMPLETE,publishers/google/models/gemini-1.5-pro,BLOCK_NONE,Article: {context}. Complete this task: {instr...,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,5.0,0.000000,0.447214,1.095445,4.2,5.0,4.8,0.300540,0.040114
2,eval-custom-metric,599561ac-d7cb-41f5-a520-420f70aea802,system.ExperimentRun,COMPLETE,publishers/google/models/gemini-1.5-pro,BLOCK_NONE,Instruction: {instruction}. Article: {context}...,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,5.0,0.447214,0.447214,0.547723,4.6,4.8,4.8,0.355634,0.069161
